Preparing dataframe for analysis:

In [ ]:
import pandas as pd

df_fb = pd.read_csv('CollegeFootballRankings24to20.csv') # Football dataframe
df_wk = pd.read_csv('WeekbyWeekRankings2020to2025.csv') # AP ek rankings dataframe

# Create Unique Date Key based on all games played from 2020 to 2024
df_fbwk = df_fb[['season','date']]
df_fbwk['date_key'] = df_fbwk['date'].astype(str) + ', '+ df_fbwk['season'].astype(str)
df_fbwk['date_key'] = pd.to_datetime(df_fbwk['date_key'])
df_fbwk.drop_duplicates(inplace=True)
df_fbwk = df_fbwk.sort_values(by='date_key', ascending=True)
#print(df_fbwk)

# Find the corresponding week number sequence for each date_key within each Season
df_ss = df_fbwk.groupby('season', as_index=False)['date_key'].min() # Find season start date
df_fbwk = pd.merge(df_fbwk, df_ss, on=['season'], how='left', suffixes=('','_season_start'))
df_fbwk['days_from_start'] = (df_fbwk['date_key'] - df_fbwk['date_key_season_start']).dt.days
df_fbwk['week_number'] = (df_fbwk['days_from_start'] // 7) + 1
#df_fbwk.head()

# Merge week number into the Football dataframe
df_fb['date'] = pd.to_datetime(df_fb['date'].astype(str) + ', ' + df_fb['season'].astype(str))

df_fbdata = pd.merge(
    df_fb,
    df_fbwk[['season','date_key','week_number']],
    left_on=['season','date'],
    right_on=['season','date_key'],
    how='left'
).drop(columns=['date_key'])

# Merge AP Week Rankings into the Football dataframe
df_fbdatafinal = pd.merge(
    df_fbdata,
    df_wk,
    left_on=['Team','season','week_number'],
    right_on=['Team','year','Week'],
    how='left'
)
df_wk.info()
# For Model Building, I would need scikit-learn, xgboost, tensorflow

df_fbdata['season'] = df_fbdata['season'].astype(int)
df_fbdata['week_number'] = df_fbdata['week_number'].astype(int)
df_wk['year'] = df_wk['year'].astype(int)
df_wk['Week'] = df_wk['Week'].astype(int)
df_wk

team_check = "Alabama"
# Questions --> Do we need to adjust all week AP rankings to reflect the final ranking of the previous / upcoming week? Do Team Names match as well?

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1975 entries, 0 to 1974
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   AP_rank  1975 non-null   int64 
 1   Team     1975 non-null   object
 2   Week     1975 non-null   int64 
 3   year     1975 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 61.8+ KB
Empty DataFrame
Columns: [Team, date, opponent, win_loss, pass, rush, rec, points_allowed, points_scored, point_differential, home_game, season, CONF, SOR, FPI, AP/CFP, SOS, GC, AVGWP, week_number]
Index: []


C:\Users\martchfr\AppData\Local\Temp\1\ipykernel_23588\1110949815.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fbwk['date_key'] = df_fbwk['date'].astype(str) + ', '+ df_fbwk['season'].astype(str)
C:\Users\martchfr\AppData\Local\Temp\1\ipykernel_23588\1110949815.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fbwk['date_key'] = pd.to_datetime(df_fbwk['date_key'])
C:\Users\martchfr\AppData\Local\Temp\1\ipykernel_23588\1110949815.py:10: SettingWithCopyWarning: 
A value is trying to be set on a